- wrap_function表示传入的函数lsp_func()一个别名一样，共享lsp_func()的传入参数，这样的话，我们可以把中间结果加入到auth这个参数中，继续传给lsp_func()

- 之后，我们修改了传入参数auth，继续给function(*args, **auth)。




In [1]:

def check_permission(user_roles):
    def decorator(function):
        @wraps(function)
        def wrap_function(*args, **auth):
            print(f'auth 0 = {auth}')
            auth['y'] = 10
            return function(*args, **auth)
        return wrap_function
    return decorator


from functools import wraps

@check_permission(['a', 'b'])
def lsp_func(**auth):
    print(f'auth 2 = {auth}')
    return auth

res = lsp_func(x=9)
res

auth 0 = {'x': 9}
auth 2 = {'x': 9, 'y': 10}


{'x': 9, 'y': 10}